Experimenting and defining functions to cleanly call APIs. A lot of the information that I want needs either a summoner ID or an account ID, not the summoner name. Functions will take a summoner name, another argument or two, and return the requested information. 

The notebook will be for testing out functions. Once functions are written and I am happy with the results, they will be moved to a normal .py script in order to be imported in other scripts. 

For quick reference and access:

[Link to Riot Developer home page](https://developer.riotgames.com/)

[Link to list of Riot APIs](https://developer.riotgames.com/apis)


### Defining the environment

In [2]:
import requests
import config
from QueueDecode import queue_decoder

In [3]:
api_key = config.api_key

In [35]:
SummonerName = 'NaiveNarwhal'

# Get Account ID

In [6]:
def get_accountID(SummonerName):
    URL = "https://na1.api.riotgames.com/lol/summoner/v4/summoners/by-name/" + SummonerName + "?api_key=" + api_key
    response=requests.get(URL)
    id_dict = response.json()
    accountId = id_dict['accountId']
    return accountId

In [7]:
get_accountID(SummonerName)

'AXDfQk149UId6QUeX1QiyU264ahFJQNE5u5Cj525ki3yiQ'

# Get Summoner ID

In [8]:
def get_summonerID(SummonerName):
    URL = "https://na1.api.riotgames.com/lol/summoner/v4/summoners/by-name/" + SummonerName + "?api_key=" + api_key
    response=requests.get(URL)
    id_dict = response.json()
    summonerId = id_dict['id']
    return summonerId

In [9]:
get_summonerID(SummonerName)

'3ScDYw3o5FOZ7sXwUGqxibkhlUf_P2vBm6RpG-pzhIupCyc'

# Get Last Match ID

In [10]:
def get_last_matchID(SummonerName):
    accountId = str(get_accountID(SummonerName))
    URL = "https://na1.api.riotgames.com/lol/match/v4/matchlists/by-account/" + accountId + "?api_key=" + api_key
    response=requests.get(URL)
    match_dict = response.json()
    # note that gameId == matchId for other Riot API calls
    gameId = match_dict['matches'][1]['gameId']
    return gameId

In [11]:
get_last_matchID(SummonerName)

3199931644

# Get Last Match Info

In [26]:
def get_last_match_info(SummonerName):
    matchID = str(get_last_matchID(SummonerName))
    
    URL = "https://na1.api.riotgames.com/lol/match/v4/matches/" + matchID + "?api_key=" + api_key
    response = requests.get(URL)
    match_info_dict = response.json()
    
    # Getting player number for the input SummonerName
    player_number_list = list(range(0,10))
    for x in player_number_list:
        summonerID = match_info_dict['participantIdentities'][x]['player']['summonerName']
        if summonerID == SummonerName:
            participantId = match_info_dict['participantIdentities'][x]['participantId']
            participant_index_num = x
            
            #returning if last game was win or loss
            if match_info_dict['participants'][x]['stats']['win'] == True:
                win = 'Win'
            elif match_info_dict['participants'][x]['stats']['win'] == False:
                win = 'Loss'
            else:
                win = 'PepeHands'
                
            kda = [match_info_dict['participants'][x]['stats']['kills'],
                   match_info_dict['participants'][x]['stats']['deaths'],
                   match_info_dict['participants'][x]['stats']['assists']] 
            
            totalDamageDealt = match_info_dict['participants'][x]['stats']['totalDamageDealt']
            
            game_mode = match_info_dict['gameMode']
            
            # getting game type with an external queueID reference json file
            queueID = match_info_dict['queueId']
            for x in queue_decoder:
                for key,value in x.items():
                    if value == queueID:
                        game_description = x['description']
                        
            print((f'{win}\n') + 
                  (f'k/d/a = {kda[0]}/{kda[1]}/{kda[2]}\n') + 
                  (f'Total Damage Dealt = {totalDamageDealt}\n') +
                  (f'{game_description}'))
                        
        else:
            return 'Pepehands'
    
    



In [32]:
get_last_match_info('Shlotchky')

'Pepehands'

# Get Rank and LP

Return the current rank, LP, and a recent 5 game history. This will allow us to watch how everybody's LP is tracking. 

In [16]:
def get_rank_history(SummonerName):
    summonerID = get_summonerID(SummonerName)
    
    URL = "https://na1.api.riotgames.com/lol/league/v4/entries/by-summoner/" + summonerID + "?api_key=" + api_key
    response = requests.get(URL)
    summoner_profile_dict = response.json()
    
    tier   = summoner_profile_dict[0]['tier']
    rank   = summoner_profile_dict[0]['rank']
    LP     = summoner_profile_dict[0]['leaguePoints']
    wins   = summoner_profile_dict[0]['wins']
    losses = summoner_profile_dict[0]['losses']
    
    print((f'{SummonerName}\n') +
          (f'{tier} {rank}\n') +
          (f'{LP} LP\n') +
          (f'{wins}/{losses}'))

In [36]:
get_rank_history(SummonerName)

NaiveNarwhal
PLATINUM IV
0 LP
120/109
